In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from aac import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "austin"
shelter = AnimalShelter(username, password, 54187)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': 200,
                          'width': 200,
                         },
                    alt="Grazioso Salvare Logo. A red line drawing of a dog")],
                    href='https://www.snhu.edu')
    ),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Footer(html.H5(id='name', children="© 2021 Jennifer Gilroy, CS-340-H6993")),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        #Dropdown list to filter animals
        dcc.Dropdown(
        id='filter-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Diaster Rescue'},
            {'label': 'All Records', 'value': 'All Records'}
        ],
            #value='All Records',
        clearable=False,
        placeholder="Select to filter / unfilter"
    )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable='single',
        selected_rows=[],
        selected_columns=[],
        page_current=0,
        page_size=20,
    ),
        
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])



#############################################
# Interaction Between Components / Controller
#############################################

#Callback for Dropdown Filtering List
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-dropdown', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries                 
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read(
            {"animal_type":"Dog",
             "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif value == 'Wilderness Rescue':
        df = pd.DataFrame.from_records(shelter.read(
            {"animal_type":"Dog",
             "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif value == 'Diaster Rescue':
        df = pd.DataFrame.from_records(shelter.read(
           {"animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    
    return (data,columns)


#Callback for Highlighting Cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Callback for Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
    #Input('datatable-id', "selected_rows")])

def update_graphs(data):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    dff = pd.DataFrame.from_dict(data)
    fig=px.pie(dff,
                 names='breed',
                 title="Breeds Available by Rescue Type"
                 )
    fig.update_traces(textposition='inside')
    return [
        dcc.Graph(
            id='graph=id',
            figure=fig
        )
    ]


          

#Callback for Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])
def update_map(viewData, row):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
        dff = pd.DataFrame.from_dict(viewData)
        #lat = (dff.iloc[row,13]).astype(float)
        #long = (dff.iloc[row,14]).astype(float)
        # Austin TX is at [30.75,-97.48] ~ corrected to [30.27,-97.73]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.27,-97.73], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.27,-97.73],
                          children=[
                              dl.Tooltip(dff.iloc[row,4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row,9]),
                                  
                              ])
                          ])
            ])
        ]
    
    


app